This program is designed to allow the user enter a series of employee id numbers, status(full-time or part-time) and a date range to create employee work schedules. The final schedule will be uploaded into paycom(HR & Payroll Management System) in csv format. This program organizes the data (Employee ID numbers, dates, times, punch type) into a certain structure needed to successfully upload it into the paycom system.    

In [ ]:
import pandas as pd
from pandas.tseries.offsets import DateOffset
from datetime import datetime
from datetime import timedelta 

Inputs for full-time employees

In [ ]:
a = list() 
while(True):
    x = input("Enter a full-time employee ID. Input 'Done' when complete: ")
    if x != "Done":
         a.append(x)
    else:
        break
idlen = len(a) #length of ID list 

Enter a full-time employee ID. Input 'Done' when complete: 2365
Enter a full-time employee ID. Input 'Done' when complete: 12014
Enter a full-time employee ID. Input 'Done' when complete: 8569
Enter a full-time employee ID. Input 'Done' when complete: 32
Enter a full-time employee ID. Input 'Done' when complete: 20
Enter a full-time employee ID. Input 'Done' when complete: Done


In [ ]:
dates = pd.date_range(start=input("Start Date: "),end= input("End Date: "), freq = 'B').to_pydatetime().tolist()
odate = len(dates) #Length of Date Range 

Start Date: 07/01/2021
End Date: 10/01/2021


Create each column using for loops

In [ ]:
#IDS
for i in range(0,odate*idlen*4-idlen): #Need to duplicate each ID of employee 4 times * and based off length of date range.
    a.append(a[i]) #create list of ids
ident = pd.Series(a) #convert list to series
ident = ident.sort_values().reset_index(drop = True) 

In [ ]:
#TIME VALUES
t = ["08:00", "12:00", "12:30", "16:30"]
for i in range(0, (odate*idlen*4)-4): #Need to duplicate t values 4 times * and based off length of date range.
    t.append(t[i]) #create list that repeats itself based off date range and number of IDs
times = pd.Series(t) #convert to Series

In [ ]:
#DATE VALUES
date_Series = pd.Series(dates)
rt = date_Series.repeat(4) #Repeated date series four times to account for 4 total punches through each day 
rt.reset_index(drop = True)
list_l = list(rt)
for i in range(0,len(list_l)*(idlen-1)):
    list_l.append(list_l[i])
date_column = pd.Series(list_l)    

In [ ]:
#PUNCHES
new_punch = ["ID","OL","IL","OD"]
for i in range(0,(odate*idlen*4)-4):
    new_punch.append(new_punch[i])
punches = pd.Series(new_punch)

Create dataframe for full-time Employees

In [ ]:
blank = pd.Series() #use this to insert for blank Series in column position 1
df = pd.concat([ident, date_column, times, punches], axis = 1) 
df.insert(loc = 1, column = "Blank", value = "") #insert blank column

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


Part-Time Employee dataframe creation

In [ ]:
v = list() #Makes list of all ID's of part-time employees 
while(True):
    list_partime = input("Enter part-time employee ID(s). Input 'Done' when done: ")
    if list_partime != "Done":
         v.append(list_partime)
    else:
        break
        

Enter part-time employee ID(s). Input 'Done' when done: 12
Enter part-time employee ID(s). Input 'Done' when done: Done


In [ ]:
# Part_Time Employee ID 

l = 2*odate #length of date range 
len(v) #length of part-time employee id list 
id_list = []

for i in v: #for loop will go through list of employee IDs
  for j in range(0,odate*2): #will iterate length of date range * 2 that will provide total number of ids to duplicate 
    id_list.append(i) #writes ids into a list

ID_Series = pd.Series(id_list) #convert list into a Series 


In [ ]:
time_list = ["8:00","12:00"] #time Series created
empty_time_list = []
for i in range (0,len(v)*odate):#need to start index position at one after end of org dataframe 
    empty_time_list.append(time_list[0])
    empty_time_list.append(time_list[1])

time_series = pd.Series(empty_time_list).reset_index(drop = True)

In [ ]:
punch_list = ["ID","OD"] #punch Series created 
empty_punch_list = []
for i in range (0,len(v)*odate):
    empty_punch_list.append(punch_list[0])
    empty_punch_list.append(punch_list[1])
punch_series = pd.Series(empty_punch_list)
len(punch_series)

134

In [ ]:
#part-time Date Series
a = pd.Series(dates).repeat(2).reset_index(drop=True) #created time date series for #1 client 
copy_b = a.copy() #copy of Series
final_series = pd.Series() #blank series to park final series

if len(v)<2: #If the length of the employee Id list is less than 2 use a above
  final_series = a
elif len(v)>=2: #If length of employee Id list is 2 or greater need to expand time series list using for loop 
  for i in range(0,len(v)): 
    final_series = pd.concat([final_series,copy_b]) #concat the org. blank series with the copy of the original series
    final_series.reset_index(drop=True) #reset index and drop original index column

final_series.reset_index(drop=True, inplace = True) #Reset index


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


In [ ]:
final_series

0     2021-07-01
1     2021-07-01
2     2021-07-02
3     2021-07-02
4     2021-07-05
         ...    
129   2021-09-29
130   2021-09-30
131   2021-09-30
132   2021-10-01
133   2021-10-01
Length: 134, dtype: datetime64[ns]

In [ ]:
#Create Dataframe with 4 Series of part-time information 
pt_final = pd.concat([ID_Series, final_series, time_series, punch_series], axis=1)
pt_final.insert(1,column = "Blank", value = "")

In [ ]:
df_final_export = pd.concat([df, pt_final]) #combine part-time df with full-time df
df_final_export.reset_index(drop = True, inplace = True) 

Drop Specific Holidays 

In [ ]:
#DROP SPECIFIC HOLIDAYS
#drop_thanksgiving = df_final_export[df_final_export[1].isin(["02-21-2022"])].reset_index()
#ithanks = list(drop_thanksgiving["index"])
#df_final_export.drop(index = ithanks, inplace = True)
#df_final_export

In [ ]:
df_final_export.to_csv("schedules.csv", index = False, header = False) #export to csv file

In [ ]:
df_final_export.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   0       1474 non-null   object        
 1   Blank   1474 non-null   object        
 2   1       1474 non-null   datetime64[ns]
 3   2       1474 non-null   object        
 4   3       1474 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 57.7+ KB


In [ ]:
df_final_export

,0,Blank,1,2,3
0,12014,,2021-07-01,08:00,ID
1,12014,,2021-07-01,12:00,OL
2,12014,,2021-07-01,12:30,IL
3,12014,,2021-07-01,16:30,OD
4,12014,,2021-07-02,08:00,ID
...,...,...,...,...,...
1469,12,,2021-09-29,12:00,OD
1470,12,,2021-09-30,8:00,ID
1471,12,,2021-09-30,12:00,OD
1472,12,,2021-10-01,8:00,ID
